In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [ ]:
## load the train model, scaler pickel , onehot

model = load_model('model.h5')

## load pickels file
with open('scaler.pkl', 'rb') as f:
    label_scaler = pickle.load(f)

with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)

with open('onehot_encoder_geo.pkl', 'rb') as f:
    lable_encoder_geo = pickle.load(f)

In [3]:
## Example input data 
input_data = {
    "CreditScore": 600,
    "Geography": "France",
    "Gender": 'Male',
    "Age": 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [8]:
## one hot encode geography

geo_encoded = lable_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=lable_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

C:\Users\chandan kumar\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [13]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [14]:
## convert gender accoding to label encoder 
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [15]:
## concat geography onehot encoded data
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [16]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [18]:
## scale the input data

input_scaled = label_scaler.transform(input_df)

In [19]:
## predict churn data


predict_data = model.predict(input_scaled)
predict_data

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step


array([[0.01564198]], dtype=float32)

In [20]:
prediction_prob = predict_data[0][0]
prediction_prob

np.float32(0.015641976)

In [21]:
if prediction_prob > 0.5:
    print('Customer is likely to churn..')
else:
    print('Customer is not likely to churn....')

Customer is not likely to churn....
